<a href="https://colab.research.google.com/github/aserdargun/DSML101/blob/main/python/Part_1_Section_07_Scopes_Closures_and_Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PART 1: FUNCTIONAL PROGRAMMING**

## Section 07 - Scopes, Closures and Decorators

### 01 - Global and Local Scopes

In Python the `global` scope refers to the `module` scope.

The scope of a variable is normally defined by where it is (lexically) defined in the code.

In [ ]:
a = 10

In this case, `a` is defined inside the main module, so it is a global variable.

In [ ]:
def my_func(n):
    c = n ** 2
    return c

In this case, `c` was defined inside the function `my_func`, so it is `local`  to the function `my_func`. In this example, `n` is also `local` to `my_func`

Global variables can be accessed from any inner scope in the module, for example:

In [ ]:
def my_func(n):
    print('global:', a)
    c = a ** n
    return c

In [ ]:
my_func(2)

global: 10


100

As you can see, `my_func` was able to reference the global variable `a`.

But remember that the scope of a variable is determined by where it is assigned. In particular, any variable defined (i.e. assigned a value) inside a function is local to that function, even if the variable name happens to be global too!

In [ ]:
def my_func(n):
    a = 2
    c = a ** 2
    return c

In [ ]:
print(a)
print(my_func(3))
print(a)

10
4
10


In order to change the value of a global variable within an inner scope, we can use the `global` keyword as follows:

In [ ]:
def my_func(n):
    global a
    a = 2
    c = a ** 2
    return c

In [ ]:
print(a)
print(my_func(3))
print(a)

10
4
2


As you can see, the value of the global variable `a` was changed within `my_func`.

In fact, we can create global variables from within an inner function - Python will simply create the variable and place it in the global scope instead of the local scope:

In [ ]:
def my_func(n):
    global var
    var = 'hello world'
    return n ** 2

Now, `var` does not exist yet, since the function has not run:

In [ ]:
print(var)

NameError: name 'var' is not defined

Once we call the function though, it will create that global `var`:

In [ ]:
my_func(2)

4

In [ ]:
print(var)

hello world


---

**BE CAREFUL!**

*Remember that whenever you assign a value to a variable without having specified the variable as `global`, it is `local` in the current scope. Moreover, it does not matter where the assignment in the code takes place, the variable is considered local in the entire scope - Python determines the scope of objects at compile-time, nor at run-time.*

*Let's see an example of this:*

In [ ]:
a = 10
b = 100

In [ ]:
def my_func():
    print(a)
    print(b)

In [ ]:
my_func()

10
100


So, this works as expected - `a` and `b` are taken from the global scope since they are referenced before being assigned a value in the local scope.

But now consider the following example:

In [ ]:
a = 10
b = 100

def my_func():
    print(a)
    print(b)
    b = 1000

In [ ]:
my_func()

10


UnboundLocalError: local variable 'b' referenced before assignment

As you can see, `b` in the line `print(b)` is considered a `local` variable - that's because the next line assigns a value to `b` - hence `b` is scoped as local by Python for the entire function.

Of course, functions are also objects, and scoping applies equally to function objects too. For example, we can "mask" the built-in `print` Python function:

In [ ]:
print = lambda x: 'hello {0}'.format(x)

def my_func(name):
    return print(name)

my_func('world')

'hello world'

You may be wondering how we get our real `print` function back!

In [ ]:
del print

In [ ]:
print('hello')

hello


Yay!!

If you have experience in some other programming languages you may ve wondering if loops and other code "blocks" haver their own local scope too. For example in Java, the following would not work:

```
for (int i=0; i<10; i++) {
    int x = 2 * i;
}
system.out.println(x);
```

But in Python it works perfectly fine:

In [ ]:
for i in range(10):
    x = 2 * i
print(x)

18


In this case, when we assigned a value to `x`, Python put it in the global (module) scope, so we can reference it after the `for` loop has finished running.

### 02 - Nonlocal Scopes

Functions defined inside another function can reference variables from that enlosing scope, just like functions can reference variables from the global scope.

In [ ]:
def outer_func():
    x = 'hello'

    def inner_func():
        print(x)
    
    inner_func()

In [ ]:
outer_func()

hello


in fact, any level of nesting is supported since Python just keeps looking in enclosing scopes until it finds what it needs (or fails to finde it by the time it finishes looking in the built-in scope, in which case a runtime error occurrs.)

In [ ]:
def outer_func():
    x = 'hello'
    def inner1():
        def inner2():
            print(x)
        inner2()
    inner1()

In [ ]:
outer_func()

hello


But if we assign a value to a variable, it is considered part of the local scope, and potentially masks enclosing scope variable names:

In [ ]:
def outer():
    x = 'hello'
    def inner():
        x = 'python'
    inner()
    print(x)

In [ ]:
outer()

hello


As you can see, `x` in outer was not changed.

To achieve this, we can use the `nonlocal` keyword:

In [ ]:
def outer():
    x = 'hello'
    def inner():
        nonlocal x
        x = 'python'
    inner()
    print(x)

In [ ]:
outer()

python


Of course, this can work at any level as well:

In [ ]:
def outer():
    x = 'hello'

    def inner1():
        def inner2():
            nonlocal x
            x = 'python'
        inner2()
    inner1()
    print(x)

In [ ]:
outer()

python


How far Python looks up the chain depends on the first occurence of the variable name in an enclosing scope.

Consider the following example:

In [ ]:
def outer():
    x = 'hello'
    def inner1():
        x = 'python'
        def inner2():
            nonlocal x
            x = 'monty'
        print('inner1 (before):', x)
        inner2()
        print('inner2 (after):', x)
    inner1()
    print('outer:', x)

In [ ]:
outer()

inner1 (before): python
inner2 (after): monty
outer: hello


What happened here, is that `x` in `inner1` masked `x` in `outer`. But `inner2` indicated to Python that `x` was nonlocal, so the first local variable up in the enclosing scope chain Python found was the one in `inner1`, hence `x` in `inner2` is actually referencing `x` that is local to `inner1`

We can change this behavior by making the variable `x` in `inner` nonlocal as well:

In [ ]:
def outer():
    x = 'hello'
    def inner1():
        nonlocal x
        x = 'python'
        def inner2():
            nonlocal x
            x = 'monty'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer:', x)

In [ ]:
outer()

inner1 (before): python
inner1 (after): monty
outer: monty


In [ ]:
x = 100
def outer():
    x = 'python'  # masks global x
    def inner1():
        nonlocal x  # refers to x in outer
        x = 'monty' # changed x in outer scope
        def inner2():
            global x  # refers to x in global scope
            x = 'hello'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer', x)  

In [ ]:
outer()
print(x)

inner1 (before): monty
inner1 (after): monty
outer monty
hello


---
**BE CAREFUL!**

*But this will not work. In `inner` Python is looking for a local variable called `x`. `outer` has a label called `x`, but it is a global variable, not local one - hence Python does not find a local variable in the scope chain.*

In [ ]:
x = 100
def outer():
    global x
    x = 'python'

    def inner():
        nonlocal x
        x = 'monty'
    inner()

SyntaxError: no binding for nonlocal 'x' found (766560350.py, line 7)

### 03 - Closures

Let's examine that concept of a cell to create an indirect reference for variables that are in multiple scopes.

In [ ]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [ ]:
fn = outer()

In [ ]:
fn.__code__.co_freevars

('x',)

As we can see, `x` is a free variable in the closure.

In [ ]:
fn.__closure__

(<cell at 0x000001C71A4F3BB0: str object at 0x000001C715E97B30>,)

Here we see that the free variable `x` is actually a reference to a cell object that is itself a reference to a string object.

Let's see what the memory address of `x` is in the outer function and the inner function. To be sure string interning does not play a role, I am going to use an object that we know Python will not automatically intern, like a list.

In [ ]:
def outer():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    def inner():
        print('inner:', hex(id(x)))
        print(x)
    return inner

In [ ]:
fn = outer()

outer: 0x1c71a3f1340


In [ ]:
fn.__closure__

(<cell at 0x000001C71A4FB730: list object at 0x000001C71A3F1340>,)

In [ ]:
fn()

inner: 0x1c71a3f1340
[1, 2, 3]


As you can see, each the memory address of `x` in `outer`, `inner` and the cell all point to the same object.

**Modifying the Free Variable**

We know we can modify nonlocal variables by using the `nonlocal` keyword. So the following will work:

In [ ]:
def counter():
    count = 0 # local variable

    def inc():
        nonlocal count # this is the count variable in counter
        count += 1
        return count
    return inc

In [ ]:
c = counter()

In [ ]:
c()

1

In [ ]:
c()

2

**Shared Extended Scopes**

As we saw in the lecture, we can set up nonlocal variables in different inner functions that reference the same outer scope variable, i.e. we have a free variable that is shared between two closure. This works because both non local variables and the outer local variable all point back to the same cell object.

In [ ]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count

    def inc2():
        nonlocal count
        count += 1
        return count

    return inc1, inc2

In [ ]:
fn1, fn2 = outer()

In [ ]:
fn1.__closure__, fn2.__closure__

((<cell at 0x000001C71A422A60: int object at 0x000001C7135A6910>,),
 (<cell at 0x000001C71A422A60: int object at 0x000001C7135A6910>,))

As you can see here, the `count` label points to the same cell.

In [ ]:
fn1()

1

In [ ]:
fn1()

2

In [ ]:
fn2()

3

**Multiple Instances of Closures**

Recall that every time a function is called, a new local scope is created.

In [ ]:
from time import perf_counter

def func():
    x = perf_counter()
    print(x, id(x))

In [ ]:
func()

21.4398551 1954650312240


In [ ]:
func()

21.4776457 1954650308912


The same thing happens with closures, they have their own extended scope every time the closure is created:

In [ ]:
def pow(n):
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    return inner

In this example, `n`, in the function `inner` is a free variable, so we have a closure that contains `inner` and the free variable `n`

In [ ]:
square = pow(2)

In [ ]:
square(5)

25

In [ ]:
cube = pow(3)

In [ ]:
cube(5)

125

We can see that the cell used for the free variable in both cases is different:

In [ ]:
square.__closure__

(<cell at 0x000001C71A422E80: int object at 0x000001C7135A6950>,)

In [ ]:
cube.__closure__

(<cell at 0x000001C71880CC10: int object at 0x000001C7135A6970>,)

In fact, these functions (`square` and `cube`) are not the same functions, even though they were "created" from the same `power` function:

In [ ]:
id(square), id(cube)

(1954649788768, 1954649790208)

---

**BE CAREFUL**

*Remember when I said the captured variable is a reference established when the closure is created, but the value is looked up only once the function is called?*

*This can create very subtle bugs in your program.*

*Consider the following example where we want to create some functions that can add 1, 2, 3, 4 and to whatever is passed to them.*

*We could do the following:*

In [ ]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [ ]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [ ]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

But suppose we want to get a little fancier and do it as follows:

In [ ]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [ ]:
adders = create_adders()

Now technically we have 4 functions in the `adders` list:

In [ ]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

The first one should add 1 to the value we pass it, the second should add 2, and so on.

In [ ]:
adders[3](10)

14

Yep! that works for the 4rh function.

In [ ]:
adders[0](10)

14

Uh Oh - what happened? In fact we get the same behavior from every one of those functions:

In [ ]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(14, 14, 14, 14)

Remember what I said about when the variable is captured and when the value is looked up?

when the lambdas are created their `n` is the `n` used in the loop - the same `n` !!


In [ ]:
adders[0].__code__.co_freevars

('n',)

In [ ]:
adders[0].__closure__

(<cell at 0x000001C71A4F35B0: int object at 0x000001C7135A6990>,)

In [ ]:
adders[1].__closure__

(<cell at 0x000001C71A4F35B0: int object at 0x000001C7135A6990>,)

In [ ]:
adders[2].__closure__

(<cell at 0x000001C71A4F35B0: int object at 0x000001C7135A6990>,)

In [ ]:
adders[3].__closure__

(<cell at 0x000001C71A4F35B0: int object at 0x000001C7135A6990>,)

So, by the time we call `adder[i]`, the free variable `n` (shared between all adders) is set to 4.

In [ ]:
hex(id(4))

'0x1c7135a6990'

As we can see the memory address of the singleton integer 4, is what that cell is point to.

If you want to use a loop to do this and not end up using the same cell for each of the free variables, we can use a simple trick that forces the evaluation of `n` at the time the closure is created, instead of when the closure function is evaluated.

We can do this by creating a parameter for `n` in our lambda whose default value is the current value of `n` - remember from an earlier video that parameter defaults are evaluated when the function is created, not called.

In [ ]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders

In [ ]:
adders = create_adders()

In [ ]:
adders[0].__closure__

Why aren't we getting anything in the closure? What about free variables?

In [ ]:
adders[0].__code__.co_freevars

()

Hmm, nothing either... Why?

Well, look at the lambda in that loop. Does it reference the variable `n` (other than in the default value)?

No. Hence, `n` is not a free variable in this case, and our lambda is just a plain lambda, not a closure.

And this code will now work as expected:

In [ ]:
adders[0](10)

11

In [ ]:
adders[1](10)

12

In [ ]:
adders[2](10)

13

In [ ]:
adders[3](10)

14

You just understand that since the default values are evaluated when the function (lambda in this case) is created, the then-current `n` value is assigned to the local variable `step`. So `step` will not change every time the lambda is called, and since `n` is not referenced inside the function (and therefore evaluated when the lambda is called), `n` is not a free variable.

**Nested Closures**

We can also nest closures, as can be seen in this example:

In [ ]:
def incrementer(n):
    def inner(start):
        current = start
        def inc():
            a = 10 # local var
            nonlocal current
            current += n
            return current
        return inc
    return inner

In [ ]:
fn = incrementer(2)

In [ ]:
fn

<function __main__.incrementer.<locals>.inner(start)>

In [ ]:
fn.__code__.co_freevars

('n',)

In [ ]:
fn.__closure__

(<cell at 0x000001C71A422BE0: int object at 0x000001C7135A6950>,)

In [ ]:
inc_2 = fn(100)

In [ ]:
inc_2

<function __main__.incrementer.<locals>.inner.<locals>.inc()>

In [ ]:
inc_2.__closure__

(<cell at 0x000001C71A4F3850: int object at 0x000001C7135D55D0>,
 <cell at 0x000001C71A422BE0: int object at 0x000001C7135A6950>)

Here you can see that the second free variable `n`, is pointing to the same cell as the free variable in `fn`.

Note that `a` is a local variable, and is not considered a free variable.

And we can call the closures as follows:

In [ ]:
inc_2()

102

In [ ]:
inc_2()

104

In [ ]:
inc_3 = incrementer(3)(200)

In [ ]:
inc_3()

203

In [ ]:
inc_3()

206

### 04 - Closure Applications - Part 1

In this example we are going to build an averager function that can average multiple values.

The twist is that we want to simply be able to feed numbers to that function and get a running average over time, not average a list which requires performing the same calculations (sum and count) over and over again.

In [ ]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [ ]:
a = Averager()

In [ ]:
a.add(10)

10.0

In [ ]:
a.add(20)

15.0

In [ ]:
a.add(30)

20.0

We can do this using a closure as follows:

In [ ]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [ ]:
a = averager()

In [ ]:
a(10)

10.0

In [ ]:
a(20)

15.0

In [ ]:
a(30)

20.0

Now, instead of storing a list and recalculating `total` and `count` every yime we need the new average, we are going to store running total and count and update each value each time a new value is added to the running average, and then return `total / count`.

Let's start with a class approach first, where we will use instance variables to store the running total and count and provide an instance method to add a new number and return the current average.

In [ ]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0

    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

In [ ]:
a = Averager()

In [ ]:
a.add(10)

10.0

In [ ]:
a.add(20)

15.0

In [ ]:
a.add(30)

20.0

Now, let's see how we might use a closure to achieve the same thing.

In [ ]:
def averager():
    total = 0
    count = 0

    def add(value):
        nonlocal total, count
        total +=  value
        count += 1
        return 0 if count == 0 else total / count
        
    return add

In [ ]:
a = averager()

In [ ]:
a(10)

10.0

In [ ]:
a(20)

15.0

In [ ]:
a(30)

20.0

**Generalizing this example**

We saw that we were essentially able t o convert a class to an equivalent functionality using closures. This is actually true in a much more general sense - very often, classes that  define a single method (other than initializers) can be implemented using a closure instead.

Let's look at another example of this.

Suppose we want something that keep track of the running elapsed time in seconds.

In [ ]:
from time import perf_counter

In [ ]:
class Timer:
    def __init__(self):
        self._start = perf_counter()

    def __call__(self):
        return (perf_counter() - self._start)

In [ ]:
a = Timer()

Now wait a bit before running the next line of code:

In [ ]:
a()

0.08975990000000067

Let's start another "timer":

In [ ]:
b = Timer()

In [ ]:
print(a())
print(b())

0.23507190000000122
0.04213780000000256


Now let's rewrite this using a closure instead:

In [ ]:
def timer():
    start = perf_counter()

    def elapsed():
        # we don't even need to makee start nonlocal
        # since we are only reading it
        return perf_counter() - start
    
    return elapsed

In [ ]:
x = timer()

In [ ]:
x()

0.03932700000000011

In [ ]:
y = timer()

In [ ]:
print(x())
print(y())

0.1200429000000014
0.04027549999999991


In [ ]:
print(a())
print(b())
print(x())
print(y())

0.5292507999999998
0.33632040000000174
0.15672759999999997
0.07672919999999905


### 05 - Closure Applications - Part 2

**Example 1**

Let's write a small function that can increment a counter for us - we don't have an incrementor in Python (the ++ operator in Java or C++ for example):

In [ ]:
def counter(initial_value):
    # initial_value is a local variable here

    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [ ]:
counter1 = counter(0)

In [ ]:
print(counter1(0))

0


In [ ]:
print(counter1())

1


In [ ]:
print(counter1())

2


In [ ]:
print(counter1(8))

10


In [ ]:
counter2 = counter(1000)

In [ ]:
print(counter2(0))

1000


In [ ]:
print(counter2(1))

1001


In [ ]:
print(counter2())

1002


In [ ]:
print(counter2(220))

1222


As you can see, each closure maintains a reference to the initial_value variable that was created when `counter` function was called - each time that function was called, a new local variable initial_value was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

**Example 2**

Let's modify this example to now build something that can run, and maintain a count of how many yimes we have run som function.

In [ ]:
def counter(fn):
    cnt = 0 # initially fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
        
    return inner

In [ ]:
def add(a, b):
    return a + b

In [ ]:
counted_add = counter(add)

And the free variables are:

In [ ]:
counted_add.__code__.co_freevars

('cnt', 'fn')

We can now call the `counted_add` function:

In [ ]:
counted_add(1, 2)

add has been called 1 times


3

In [ ]:
counted_add(2, 3)

add has been called 2 times


5

In [ ]:
def mult(a, b, c):
    return a * b * c

In [ ]:
counted_mult = counter(mult)

In [ ]:
counted_mult(1, 2, 3)

mult has been called 1 times


6

In [ ]:
counted_mult(2, 3, 4)

mult has been called 2 times


24

**Example 3**

Let's take this one step further, and actually store the function name and the number of calls in a global dictionary instead of just printing it out all the time.

In [ ]:
counters = dict()

def counter(fn):
    cnt = 0 # initially fn has been run zero times

    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt # counters is global
        return fn(*args, **kwargs)
        
    return inner

In [ ]:
counted_add = counter(add)
counted_mult = counter(mult)

Note that `counters` is a `global` variable, and therefore not a free variable:

In [ ]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [ ]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

We can now call out functions:

In [ ]:
counted_add(1, 2)

3

In [ ]:
counted_add(2, 3)

5

In [ ]:
counted_mult(1, 2, 'a')

'aa'

In [ ]:
counted_mult(2, 3, 'b')

'bbbbbb'

In [ ]:
counted_mult(1, 1, 'abc')

'abc'

In [ ]:
print(counters)

{'add': 2, 'mult': 3}


Of course this relies on us creating the counters global variable first and making sure we are naming it that way, so instead, we're going to pass it as an argument to the counter function:

In [ ]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

In [ ]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [ ]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

As you can see, counters is now a free variable.

We can now call our functions:

In [ ]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [ ]:
print(func_counters)

{'add': 5, 'mult': 10}


Of course, we don't have to assign the "counted" version of our functions a new name - we can simply assign it to the same name!

In [ ]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [ ]:
fact = counter(fact, func_counters)

In [ ]:
fact(0)

1

In [ ]:
fact(3)

6

In [ ]:
fact(4)

24

In [ ]:
print(func_counters)

{'add': 5, 'mult': 10, 'fact': 3}


Notice, how we essentially added some functionality to our fact function, without modifying what the fact function actually returns.

This leads us straight into our next topic: decorators!

### 06 - Decorators - Part 1

Recall the example in the last section where we wrote a simple closure to count how many times a function had been run:

In [ ]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [ ]:
def add(a, b=0):
    """
    return the sum of a and b
    """
    return a + b

In [ ]:
help(add)

Help on function add in module __main__:

add(a, b=0)
    return the sum of a and b



Here's the memory address that `add` points to:

In [ ]:
id(add)

1954621448048

Now we create a closure using the `add` function as an argument to the `counter` function:

In [ ]:
add = counter(add)

And you'll note that `add` is no longer the same function as before. Indeed the memory address `add` points to is no longer the same:

In [ ]:
id(add)

1954621447040

In [ ]:
add(1, 2)

Function add was called 1 times


3

In [ ]:
add(2, 2)

Function add was called 2 times


4

What happened is that we put our `add` function 'through' the `counter` function - we usually say that we decorated our function add.

And we call that counter function a decorator.

There is a shorthand way of decorating our function without having to type:

`func = counter(func)`

In [ ]:
@counter
def mult(a: float, b: float=1, c:float=1) -> float:
    """
    returns the product of a, b, and c
    """
    return a * b * c

In [ ]:
mult(1, 2, 3)

Function mult was called 1 times


6

In [ ]:
mult(2, 2, 2)

Function mult was called 2 times


8

Let's do a little bit of introspection on our two decorated functions:

In [ ]:
add.__name__

'inner'

In [ ]:
mult.__name__

'inner'

As you can see, the name of the function is no longer `add` or ``mult`, but instead it is the name of that `inner` function in our decorator.

In [ ]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



As you can see, we've lost our docstring and parameter annotarions!

What about instrospecting the parameters of `add` and `mult`:

In [ ]:
import inspect

In [ ]:
inspect.getsource(add)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

In [ ]:
inspect.getsource(mult)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

Even the signature is gone:

In [ ]:
inspect.signature(add)

<Signature (*args, **kwargs)>

In [ ]:
inspect.signature(mult)

<Signature (*args, **kwargs)>

Even the parameter defaults documentation is are gone:

In [ ]:
inspect.signature(add).parameters

mappingproxy({'args': <Parameter "*args">, 'kwargs': <Parameter "**kwargs">})

In general, when we create decorated functions, we end up "losing" a lot of the metadate of our original function!

However, we can put that information back in - it can get quite complicated.

Let's see how we might be able to do that for some simple things, like the docstring and the function name.

In [ ]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

In [ ]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [ ]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    returns sum of two integers



In [ ]:
add.__name__

'add'

At least we have the docstring and function name back... But what about the parameters? Our real `add ` function takes two positional parameters, but because the closure used a generic way of accpeting ***args and ***kwargs, we lose this information.

We can use a special function in the `functools` module, called `wraps`. In fact, that function is a decorator itself!

In [ ]:
from functools import wraps

In [ ]:
def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
    
    return inner

In [ ]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [ ]:
help(add)

Help on function add in module __main__:

add(a: int, b: int = 10) -> int
    returns sum of two integers



Yay!!! Everything is back to normal.

In [ ]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returns sum of two integers\n    """\n    return a + b\n'

In [ ]:
inspect.signature(add)

<Signature (a: int, b: int = 10) -> int>

In [ ]:
inspect.signature(add).parameters

mappingproxy({'a': <Parameter "a: int">, 'b': <Parameter "b: int = 10">})

### 07 - Decorator Application - Timer

Here we go back to an example we have seen in the past - timing how long it takes to run a certain fucntion.

In [ ]:
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}{1}'.format(k,v) for (k,v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__,
                                                      args_str,
                                                      elapsed))
        return result
                   
    return inner

Let's write a function that calculates the n-th Fibanacci number:

`1, 1, 2, 3, 5, 8, ...`

We will implement this using three different methods:

1. recursion
2. a loop
3. functional programming (reduce)

We use a 1-based system, e.g. first Fibonacci number has index 1, etc.


**Using Recursion**

In [ ]:
def calc_recursive_fib(n):
    if n <=2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

In [ ]:
calc_recursive_fib(3)

2

In [ ]:
calc_recursive_fib(6)

8

In [ ]:
@timed
def fib_recursed(n):
    return calc_recursive_fib(n)

In [ ]:
fib_recursed(33)

fib_recursed(33) took 1.409342s to run.


3524578

In [ ]:
fib_recursed(34)

fib_recursed(34) took 2.101899s to run.


5702887

In [ ]:
fib_recursed(35)

fib_recursed(35) took 3.193298s to run.


9227465

There's a reason we did not decorate our recursive function directly!

In [ ]:
@timed
def fib_recursed_2(n):
    if n<=2:
        return 1
    else:
        return fib_recursed_2(n-1) + fib_recursed_2(n-2)

In [ ]:
fib_recursed_2(10)

fib_recursed_2(2) took 0.000001s to run.
fib_recursed_2(1) took 0.000001s to run.
fib_recursed_2(3) took 0.000534s to run.
fib_recursed_2(2) took 0.000001s to run.
fib_recursed_2(4) took 0.000685s to run.
fib_recursed_2(2) took 0.000001s to run.
fib_recursed_2(1) took 0.000001s to run.
fib_recursed_2(3) took 0.000133s to run.
fib_recursed_2(5) took 0.000951s to run.
fib_recursed_2(2) took 0.000001s to run.
fib_recursed_2(1) took 0.000001s to run.
fib_recursed_2(3) took 0.001983s to run.
fib_recursed_2(2) took 0.000001s to run.
fib_recursed_2(4) took 0.002256s to run.
fib_recursed_2(6) took 0.003304s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(1) took 0.000000s to run.
fib_recursed_2(3) took 0.000042s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(4) took 0.000113s to run.
fib_recursed_2(2) took 0.000000s to run.
fib_recursed_2(1) took 0.000000s to run.
fib_recursed_2(3) took 0.000267s to run.
fib_recursed_2(5) took 0.000455s to run.
fib_recursed_2(7

55

Since we are calling the function recursively, we are actually calling the **decorated** function recursively. In this case I wanted the total time to calculate the n-th number, not the time for each recursion.

You will notice from the above how inefficent the recusive method is: the same fibonacci numbers are calculated repeatedly! This is why as the value of `n` start increasing beyond 30 we start seeing considerable slow downs.

**Using a Loop**

In [ ]:
@timed
def fib_loop(n):
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2

In [ ]:
fib_loop(3)

fib_loop(3) took 0.000005s to run.


2

In [ ]:
fib_loop(6)

fib_loop(6) took 0.000004s to run.


8

In [ ]:
fib_loop(34)

fib_loop(34) took 0.000008s to run.


5702887

In [ ]:
fib_loop(35)

fib_loop(35) took 0.000007s to run.


9227465

As you can see this method is muvh more efficient!

**Using Reduce**

We first need to understand how we are going to calculate the Fibonacci sequence using reduce:

```
n=1:
(1, 0) -- > (1, 1)

n=2:
(1, 0) --> (1, 1) --> (1 + 1, 1) = (2, 1) : result = 2

n=3:
(1, 0) --> (1, 1) --> (2, 1) --> (2+1, 2) = (3, 2) : result = 3

n=4:
(1, 0) --> (1, 1) --> (2, 1) --> (3, 2) --> (5, 3) : result = 5

```
In general each step in the reduction is as follows:

```
previous value = (a, b)
new value = (a+b, a)

```
if we start our reduction with an initial value of `(1, 0)` we need to run our "loop" n times.

We therefore use a "dummy" sequence of length `n` to create `n` steps in our reduce.

In [ ]:
from functools import reduce

@timed
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n-1)
    fib_n = reduce(lambda prev, n: (prev[0] + prev[1], prev[0]),
                   dummy,
                   initial)
    return fib_n[0]

In [ ]:
fib_reduce(3)

fib_reduce(3) took 0.000008s to run.


2

In [ ]:
fib_reduce(6)

fib_reduce(6) took 0.000008s to run.


8

In [ ]:
fib_reduce(34)

fib_reduce(34) took 0.000020s to run.


5702887

In [ ]:
fib_reduce(35)

fib_reduce(35) took 0.000021s to run.


9227465

Noe we can run a quick comparison between the various timed implementations:

In [ ]:
fib_recursed(35)
fib_loop(35)
fib_reduce(35)

fib_recursed(35) took 3.273193s to run.
fib_loop(35) took 0.000006s to run.
fib_reduce(35) took 0.000014s to run.


9227465

Even though the recursive algorithm is by far the easisest to understand, it is also the slowest. We'll see how to fix this in an upcoming section using a technique called **memoization**.

First let's focus on the loop and reduce variants. Our timing is not very effective since we only time a single calculation for each - ther could be some variance if we run these tests multiple times:

In [ ]:
for i in range(10):
    result = fib_loop(1000)

fib_loop(1000) took 0.000195s to run.
fib_loop(1000) took 0.000171s to run.
fib_loop(1000) took 0.000165s to run.
fib_loop(1000) took 0.000163s to run.
fib_loop(1000) took 0.000170s to run.
fib_loop(1000) took 0.000098s to run.
fib_loop(1000) took 0.000173s to run.
fib_loop(1000) took 0.000216s to run.
fib_loop(1000) took 0.000175s to run.
fib_loop(1000) took 0.000155s to run.


In [ ]:
for i in range(10):
    result = fib_reduce(1000)

fib_reduce(1000) took 0.000379s to run.
fib_reduce(1000) took 0.000423s to run.
fib_reduce(1000) took 0.000295s to run.
fib_reduce(1000) took 0.000286s to run.
fib_reduce(1000) took 0.000517s to run.
fib_reduce(1000) took 0.000509s to run.
fib_reduce(1000) took 0.000626s to run.
fib_reduce(1000) took 0.000598s to run.
fib_reduce(1000) took 0.000343s to run.
fib_reduce(1000) took 0.000323s to run.


In general it is better to time the same function call multiple times and generate and average of the run times.

We'll see in an upcoming section how we can do this from within our decorator.

In the meantime observe that the simple loop approach seems to perform about twice as fast as the reduce approach!!

The moral of this side note is that simply because you can do something in Python using some fancy or cool technique does not mean you should!

We technically could write our reduce-based function as a one liner:

In [ ]:
from functools import reduce
fib_1 = timed(lambda n: reduce(lambda prev, n: (prev[0] + prev[1], prev[0]),
                               range(n),
                               (0, 1))[0])

In [ ]:
fib_loop(100)

fib_loop(100) took 0.000012s to run.


354224848179261915075

In [ ]:
fib_1(100)

<lambda>(100) took 0.000045s to run.


354224848179261915075

So yes, it's cool that you can write this using a single line of code, but consider two things here:

1. Is it as efficient as another method?
2. Is the code readable=

Code readability is something I cannot emphasize enough. Given similar efficiencies (cpu / memory), give preference to code that is more easily understandable!

Sometimes, if the efficiency is not greatly impacted (or does not matter in absolute terms), I might even give preference to less efficient, but mote readable (i.e. understandable), code.

But enough of the soapbox already :-)

### 08 - Decorator Application - Logger, Stacked

In this example we're going to create a utility decorator the will log function calls (to the console, but in practice you would be writing your logs to a file (e.g. using Python's built-in logger), or to a database, etc.

In [ ]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
    
    return inner

In [ ]:
@logged
def func_1():
    pass

In [ ]:
@logged
def func_2():
    pass

In [ ]:
func_1()

func_1: called 2022-11-18 16:52:30.005126+00:00


In [ ]:
func_2()

func_2: called 2022-11-18 16:52:30.021121+00:00


Now we may additionaly also want to time the function. We can certainly include the code to do so in our `logged` decorator, but we could also just use the `@timed` decorator we already wrote by **stacking** our decorators.

In [ ]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [ ]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [ ]:
factorial(10)

factorial: called 2022-11-18 16:52:30.086676+00:00
factorial ran for 0.000535s


3628800

Note that the order in which we stack the decorators can make a difference!

Remember that this is because our stacked decorators essentially amounted to:

In [ ]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

So in this case the `timed` decorator will be called first, followed by the `logged` decorator.

You may wonder why the printed output seems reversed. Look at how the decorators were defined - they first ran the function passed in, and then printed the result.

So in the above example, a simplified look at what happens in each decorator:

* `timed(fn)(*args, **kwargs):`
    1. calls `fn(*args, **kwargs)`
    2. prints timing
* `logged(fn)(*args, **kwargs):`
    1. calls `fn(*args, **kwargs)`
    2. prints log info
    
So, calling `factorial = timed(logged(factorial))`

is equivalent to:

```
fn = logged(factorial)
factorial = timed(fn)
    
factorial(n) --> call timed(fn)(n)
             --> call fn(n), then printing timing
             --> call logged(original_factorial)(n), then print timing
             --> call original_factorial(n), then log, then print timing
```

So as you can see, the `timed` decorator ran first, but it called the logged decorated function first, then printed the result - hence why the print output seems reversed.

In [ ]:
factorial(10)

factorial: called 2022-11-18 16:52:30.133233+00:00
factorial ran for 0.000235s


3628800

But in the following case, the `logged` decorator will run first, followed by the `timed` decorator:

In [ ]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = logged(timed(factorial))

In [ ]:
factorial(10)

factorial ran for 0.000013s
factorial: called 2022-11-18 16:52:30.164312+00:00


3628800

Or, using the `@` notation:

In [ ]:
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [ ]:
factorial(10)

factorial ran for 0.000013s
factorial: called 2022-11-18 16:52:30.196315+00:00


3628800

In [ ]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [ ]:
factorial(10)

factorial: called 2022-11-18 16:52:30.228311+00:00
factorial ran for 0.000453s


3628800

To make this clearer, let's write two very simple decorators as follows:

In [ ]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn()
    return inner

In [ ]:
def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

In [ ]:
@dec_1
@dec_2
def my_func():
    print('running my_func')

In [ ]:
my_func()

running dec_1
running dec_2
running my_func


---
**BE CAREFUL**

*There is a priority difference between `return result` and `return fn()`*

You may wonder whether this really matters in practice. And yes, it can.

Consider an API that contains various functions that can be called. However, endpoints are secured and can only be run by authenticated users who have some specific role(s). If they do not have the role you want to return an unauthorized error. Bur if they do, then you want to log that they called the endpoint.

In this case you may have one decorator that is used to check authentication and permissions (and immediately return an unauthorized error from the API if applicable), and the other to log the call.

If you decorated it this way:

```
@log
@authorize
def my_endpoint():
    pass
```

then the call would always be logged.

But, in this instance:

```
@authorize
@log
def my_endpoint():
    pass
```

your endpoint would only get logged if the user passed the `authorize` test.

### 09 - Decorators Application (Memoization)

Let's go back to our Fibonacci example:

In [ ]:
def fib(n):
    print('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

When we run this, we see tahat it is quite inefficient, as the same Fibonacci numbers get calculated multiple times:

In [ ]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)


8

It would be better if we could somehow "store" these results, sof if we have calculated `fib(4)` and `fib(3)` before, we could simply recall the these values when calculating `fib(5) = fib(4) + fib(3)` instead of recalculating them.

This concept of improving the efficiency of our code by caching pre-calculated values so they do not need to be re-calculated everytime, is called "memoization"

We can approach this using a simple class and a dictionary that stores any Fibonacci number that's already been calculated:

In [ ]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}
        
    def fib(self, n):
        if n not in self.cache:
            print('Calculating fib({0})'.format(n))
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [ ]:
f = Fib()

In [ ]:
f.fib(1)

1

In [ ]:
f.fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


8

In [ ]:
f.fib(7)

Calculating fib(7)


13

Let's see how we could rewrite this using a closure:

In [ ]:
def fib():
    cache = {1: 1, 2: 1}
    
    def calc_fib(n):
        if n not in cache:
            print('Calculating fib({0})'.format(n))
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    
    return calc_fib

In [ ]:
f = fib()

In [ ]:
f(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

Now let's see how we would implement this using a decorator:

In [ ]:
from functools import wraps

def memoize_fib(fn):
    cache = dict()
    
    @wraps(fn)
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    
    return inner

In [ ]:
@memoize_fib
def fib(n):
    print('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(3)

Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


2

In [ ]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)


55

In [ ]:
fib(6)

8

As you can see, we are hitting the cache when the values are available.

Now, we made our memoization decorator "hardcoded" to single argument functions - we could make it more generic.

For example, to handle an arbitrary number of positional arguments and keyword-only arguments we could do the following:

In [ ]:
def memoize(fn):
    cache = dict()
    
    @wraps(fn)
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]
    
    return inner

In [ ]:
@memoize
def fib(n):
    print('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


8

In [ ]:
fib(7)

Calculating fib(7)


13

Of course, with this rather generic memoization decorator we can memoize other functions too:

In [ ]:
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

In [ ]:
fact(5)

Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


120

In [ ]:
fact(5)

Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


120

In [ ]:
@memoize
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

In [ ]:
fact(6)

Calculating 6!
Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


720

In [ ]:
fact(6)

720

Our simple memoizer has a drawback however:

* the cache size is unbounded - probably not a good thing! In general we want to limit the cache to a certain number of entries, balancing computational efficiency vs memory utilization.

* we are not handling **kwargs

Memoization is such a common thing to do that Python actually has a memoization decorator built for us!

It's in the, you guessed it, **functools** module, and is called **lru_cache** and is going to be quite a bit more efficient compared to the rudimentary memoization example we did above.

LRU Cache = Least Recently Used caching: since the cache is not unlimited, at some point cached entries need to be discarded, and the least reacently used entries are discarded first

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache
def fact(n):
    print("Calculating fact({0})".format(n))
    return 1 if n < 2 else n * fact(n-1)

In [ ]:
fact(5)

Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


120

In [ ]:
fact(4)

24

As you can see, `fact(4)` was returned via a cached entry!

Same thing with our Fibonacci function:

In [ ]:
@lru_cache()
def fib(n):
    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


8

In [ ]:
fib(5)

5

Recall from a few lessons back that we timed the calculation for Fibonacci numbers. Calculating fib(35) took several seconds - everytime...

In [ ]:
from time import perf_counter

In [ ]:
def fib_no_memo(n):
    return 1 if n < 3 else fib_no_memo(n-1) + fib_no_memo(n-2)

In [ ]:
start = perf_counter()
result = fib_no_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 3.7892764999999997s


In [ ]:
@lru_cache()
def fib_memo(n):
    return 1 if n < 3 else fib_memo(n-1) + fib_memo(n-2)

In [ ]:
start = perf_counter()
result = fib_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 0.00018599999999935335s


And if we make the calls again:

In [ ]:
start = perf_counter()
result = fib_no_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 3.400475400000005s


In [ ]:
start = perf_counter()
result = fib_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 0.00014019999999703714s


You may have noticed that the `lru_cache` decorator was implemented using `()` - we'll see more on this later, but that's because decorators can themselves have parameters (beyond the function being decorated).

One of the arguments to the ``lru_cache` decorator is the size of the cache - it defaults to 128 itms, but we can easily change this - for performance reasons use powers of 2 for the cache size (or None for unbounded cache):

In [ ]:
@lru_cache(maxsize=8)
def fib(n):
    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [ ]:
fib(20)

Calculating fib(20)
Calculating fib(19)
Calculating fib(18)
Calculating fib(17)
Calculating fib(16)
Calculating fib(15)
Calculating fib(14)
Calculating fib(13)
Calculating fib(12)
Calculating fib(11)


6765

In [ ]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

You'll not how Python had to recalculate `fib` for `10, 9` etc. This is because the cache can only contain 10 items, so when we calculated `fib(20)`, it stored fib for `20, 19, ..., 11 ` (10 items) and therefore the oldest items fib `10, 9, ..., 1` were removed from the cache to make space.

### 10 - Decorators - Part 2

We have seen how to create some simple and not so simple decorators.

However we have also been using built-in decorators that can accept parameters, such as `wraps` and `lru_cache`.

This can be quite useful and we can accomplish the same thing ourselves.

First recall our original timer decorator from an earlier lesson (Decorator Application - Time):

In [ ]:
def timed(fn):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print('Run time: {0:.6f}s'.format(elapsed))
        return result
    
    return inner

In [ ]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

def fib(n):
    return calc_fib_recurse(n)

We can decorate our Fibonacci function using the `@` syntax, or the longer syntax as follows:

In [ ]:
fib = timed(fib)

In [ ]:
fib(30)

Run time: 0.278976s


832040

Let's modify this so the timer runs the function multiple times and calculates the average run time:

In [ ]:
def timed(fn):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / 10
        print('Avg Run time: {0:.6f}s'.format(avg_elapsed))
        return result
    
    return inner

And again we decorate it using the long syntax:

In [ ]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib)

In [ ]:
fib(28)

Avg Run time: 0.102667s


317811

But that value of 10 has been hardcoded. Let's make it a parameter instead.

In [ ]:
def timed(fn, num_reps):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s'.format(avg_elapsed,
                                             num_reps))
        return result
    
    return inner

Now to decorate our Fibonacci function we have to use the long syntax (as we saw in the lecture, the `@` syntax will not work):

In [ ]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib, 5)

In [ ]:
fib(28)

Avg Run time: 0.223914s


317811

The problem is that we cannot use the `@` decorator syntax because when using that syntax Python passes a single argument to the decorator: the function we are decorating - nothing else.

Of course we could just use what we did above, but the decorator syntax is kind of neat, so it would be nice to retain the ability to use it.

We just need to change our thinking a little bit to do this:

First, when we see the following syntax:

```
@dec 
def my_func(): 
    pass
```
We see that `dec` must be a function that takes a single argument, the function being decorated.

You'll note that `dec` is just a function, but we do not call `dec` when we decorate `my_func`, we simply use the label `dec`.

Then Python does:

`my_func = dec(my_func)`

Let's try a concrete example:

In [ ]:
def dec(fn):
    print("running dec")
    
    def inner(*args, **kwargs):
        print("running inner")
        return fn(*args, **kwargs)
    
    return inner

In [ ]:
@dec
def my_func():
    print("running my_func")

running dec


As we can see, when we decorated `my_func`, the `dec` function was called at that time.

(Because Python did this:

`my_func = dec(my_func)`

so `dec` was called)

And when we now call `my_func`, we see that the `inner` function is called, followed by the original `my_func`

In [ ]:
my_func()

running inner
running my_func


But what if `dec` was not the decorator itself, but instead created and returned a decorator?

Let's see how we might do this:

In [ ]:
def dec_factory():
    print('running dec_factory')
    def dec(fn):
        print('running dec')
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
        return inner
    return dec

So as you can see, calling `dec_generator()` will return that `dec` function which is our decorator:

In [ ]:
@dec_factory()
def my_func(a, b):
    print(a, b)

running dec_factory
running dec


You can see that both `dec_generator` and `dec` we already called.

In [ ]:
my_func(10, 20)

running inner
10 20


And there you go, all we did is basically create a decorator by calling a function (`dec_factory`) and use the return value of that call (the `dec` function) as our actual decorator:

We could have done the decoration this way too:

In [ ]:
dec = dec_factory()

running dec_factory


In [ ]:
@dec
def my_func():
    print('running my_func')

running dec


In [ ]:
my_func()

running inner
running my_func


Or even this way:

In [ ]:
dec = dec_factory()

def my_func():
    print('running my_func')
    
my_func = dec(my_func)

running dec_factory
running dec


In [ ]:
my_func()

running inner
running my_func


Of course we could even decorate it this way using a single statement:

In [ ]:
def my_func():
    print('running my_func')
    
my_func = dec_factory()(my_func)

running dec_factory
running dec


In [ ]:
my_func()

running inner
running my_func


OK, so now we have decorated our function using, not a decorator, but a decorator factory as follows:

In [ ]:
def dec_factory():
    def dec(fn):
        def inner(*args, **kwargs):
            print('running decorator inner')
            return fn(*args, **kwargs)
        return inner
    return dec

In [ ]:
@dec_factory()
def my_func(a, b):
    return a + b

In [ ]:
my_func(10, 20)

running decorator inner


30

You should note that in this approach, we are calling `dec_factory()`, note the parentheses `()`, and then using the return value (a decorator) to decorate our function.

So, we could pass arguments as we do so without affecting the final outcome. In fact we can even access them from anywhere inside `dec_factory`, including any of the nested functions!

Let's try this:

In [ ]:
def dec_factory(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print('running decorator inner')
            print('free vars: ', a, b)  # a and b are free variables!
            return fn(*args, **kwargs)
        return inner
    return dec

In [ ]:
@dec_factory(10, 20)
def my_func():
    print('python rocks')

In [ ]:
my_func()

running decorator inner
free vars:  10 20
python rocks


And this is how we can create decorators with parameters. We do not directly create a decorator, instead we use an outer function that returns a decorator when called, and pass arguments to that outer function, which the decorator and its inner function can of course access as nonlocal (free) variables.

So now, let's ho back to our original problem where we wanted our timing deocrator to run a number of loops which could be specified as a parameter when decorating the function we want to time.

Here it is again:

In [ ]:
def timed(fn, num_reps):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                        num_reps))
        return result
    
    return inner

So, all we need to do is create an outer function around our timed decorator, and pass the `num_reps` argument to that outer function instead:

In [ ]:
def timed_factory(num_reps=1):
    def timed(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return timed    

In [ ]:
@timed_factory(5)
def fib(n):
    return calc_fib_recurse(n)

In [ ]:
fib(30)

Avg Run time: 0.310106s (5 reps)


832040

Just to put the finishing touch on this, we probably don't want to have our ouer function named the way it is (`timed_factory`). Instead we probably just want to call it `timed`. So lets just do this final part:

In [ ]:
from functools import wraps

def timed(num_reps=1):
    def decorator(fn):
        from time import perf_counter

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return decorator  

In [ ]:
@timed(5)
def fib(n):
    return calc_fib_recurse(n)

In [ ]:
fib(30)

Avg Run time: 0.291452s (5 reps)


832040

### 11 - Decorator Application - Decorator Class

If you recalls how we wrote a parameterized decorator, we had to write a decorator factory that took in the arguments for our decorator and then returned the decorator (which could reference the arguments as free variables).

Very simply:

In [ ]:
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print('decorated function called: a={0}, b={1}'.format(a, b))
            return fn(*args, **kwargs)
        return inner
    return dec

In [ ]:
@my_dec(10, 20)
def my_func(s):
    print('hello {0}'.format(s))

In [ ]:
my_func('world')

decorated function called: a=10, b=20
hello world


So, our decorator factory was passed some arguments, and returned a callable which took one single parameter, the function being decorated, but also had access to the arguments passed to the factory.

Now, recall that we can make our class instances callable, simply by implementing the `__call__` method.

Here's a simple example:

In [ ]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self):
        print('MyClass instance called: a={0}, b={1}'.format(self.a, self.b))

In [ ]:
my_class = MyClass(10, 20)

In [ ]:
my_class()

MyClass instance called: a=10, b=20


So let's mddify this just a bit, and have the `__call__` method be our decorator!

In [ ]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print('MyClass instance called: a={0}, b={1}'.format(self.a, self.b))
            return fn(*args, **kwargs)
        return inner

So, we can decorate our functions this way:

In [ ]:
@MyClass(10, 20)
def my_func(s):
    print('Hello {0}!'.format(s))

Remember that `@MyClass(10, 20)` returned an object of type `MyClass`. But that object is itself callable, so we could do something line:

```
my_func = MyClass(10, 20)(my_func)
```
or, more simply

```
@MyClass(10, 20)
def my_func(s):
    print(s)
```

In [ ]:
my_func('Python')

MyClass instance called: a=10, b=20
Hello Python!


So as you can see, we can also use callable classes to decorate functions!

### 12 - Decorator Application - Decorating Classes

We have so far worked with decorating functions. This means we can decorate functions defined with a `def` statement (we can use the `@` sytnax, or the long form). Since class methods are functions, they can be decorated too. Lambda expressions can also be decorated (using the long form).

But if you think about how our decorators work, they take a single parameter, a function, and return some other function - usually a closure that uses the original function that was passed as an argument.

We could use the same concept to accept, not a function, but a class instead. We could reference that class inside our decorator, modify it, and then return that modified class.

First we look at something called **monkey patching**. It boils down to modifying or extending our code at **run time**.

For example we can modify or add attributes to classes at run time. Modules too.

In Python, many of the classes we use can be modified at run time (built-ins like strings, lists, and so on, cannot).

But  classes written in Python, such as the ones we write, and even library classes, as long as they are written in Python, not C, can. For example `Fraction` in the `fractions` module can be monkey patched.

Just because we can do something however, does not mean we should! Monkey patching can be extremely useful, but don't do it just because you can - as always there should be a real reason to do it, as we'll see in a bit.

Also, in general it is a bad idea to monkey patch the special methods `__???__` (such as `__len__`) as this will often not work due to how these methods are searched for by Python.

In [ ]:
from fractions import Fraction

In [ ]:
Fraction.speak = lambda self: 'This is a late parrot.'

In [ ]:
f = Fraction(2, 3)

In [ ]:
f

Fraction(2, 3)

In [ ]:
f.speak()

'This is a late parrot.'

Yes, this is obviously nonsense, but you get the idea that you can add attributes to classes even if you do not have direct control over class, or after your class has been defined.

If you want a more useful method, how about one that tells us if the Fraction is an integral number? (i.e. denominator is `1`)

In [ ]:
Fraction.is_integral = lambda self: self.denominator == 1

In [ ]:
f1 = Fraction(1, 2)
f2 = Fraction(10, 5)

In [ ]:
f1.is_integral()

False

In [ ]:
f2.is_integral()

True

Now, we can make this change to the class by calling a function to do it instead:

In [ ]:
def dec_speak(cls):
    cls.speak = lambda self: 'This is a very late parrot.'
    return cls

In [ ]:
Fraction = dec_speak(Fraction)

(Hopefully the above code reminds you of decorators.)

In [ ]:
f = Fraction(10, 2)

In [ ]:
f.speak()

'This is a very late parrot.'

We can use that function to decorate our custom classes too, using the short `@` syntax too.

In [ ]:
@dec_speak
class Parrot:
    def __init__(self):
        self.state = 'late'

In [ ]:
polly = Parrot()

In [ ]:
polly.speak()

'This is a very late parrot.'

Using this technique we could for example add a useful *reciprocal* attribute to the Fraction class, but of course since it would probably be a one time kind of thing (how many Fraction classes are there that you will want to add a reciprocal to after all), there's no need for decorators. Decorators are useful when they are able to be reused in more general ways.

In [ ]:
Fraction.recip = lambda self: Fraction(self.denominator, self.numerator)

In [ ]:
f = Fraction(2, 3)

In [ ]:
f

Fraction(2, 3)

In [ ]:
f.recip()

Fraction(3, 2)

These example are quite trivial, and not very useful.

So why bring this up?

Because this same technique can be used for more interesting things.

As a first example, let's say you typically like to inspect various properties of an object for debugging purposes, maybe the memory address, it's current state (property values), and the time at which the debug info was generated.

In [ ]:
from datetime import datetime, timezone

In [ ]:
def debug_info(cls):
    def info(self):
        results = []
        results.append('time: {0}'.format(datetime.now(timezone.utc)))
        results.append('class: {0}'.format(self.__class__.__name__))
        results.append('id: {0}'.format(hex(id(self))))
        
        if vars(self):
            for k, v in vars(self).items():
                results.append('{0}: {1}'.format(k, v))
        
        # we have not covered lists, the extend method and generators,
        # but note that a more Pythonic way to do this would be:
        #if vars(self):
        #    results.extend('{0}: {1}'.format(k, v) 
        #                   for k, v in vars(self).items())
        
        return results
    
    cls.debug = info
    
    return cls  

In [ ]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return 'Hello there!'

In [ ]:
p1 = Person('John', 1939)

In [ ]:
p1.debug()

['time: 2022-11-18 16:52:44.796363+00:00',
 'class: Person',
 'id: 0x1c71a4bff40',
 'name: John',
 'birth_year: 1939']

And of course we can decorate other classes this way too, not just a single class:

In [ ]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed_mph):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed_mph = top_speed_mph
        self.current_speed = 0
        
    @property
    def speed(self):
        return self.current_speed
    
    @speed.setter
    def speed(self, new_speed):
        self.current_speed = new_speed

In [ ]:
s = Automobile('Ford', 'Model T', 1908, 45)

In [ ]:
s.debug()

['time: 2022-11-18 16:52:44.842366+00:00',
 'class: Automobile',
 'id: 0x1c71a4c7310',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 0']

In [ ]:
s.speed = 20

In [ ]:
s.debug()

['time: 2022-11-18 16:52:44.872903+00:00',
 'class: Automobile',
 'id: 0x1c71a4c7310',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 20']

In [ ]:
from math import sqrt

In [ ]:
class Point:
    def __init__(self,x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return 'Point({0},{1})'.format(self.x, self.y)
    

In [ ]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [ ]:
abs(p1)

3.605551275463989

In [ ]:
p1, p2

(Point(2,3), Point(2,3))

In [ ]:
p1 == p2

False

Hmm, we probably would have expected `p1` to be equal `p2` since it has the same coordinates. But by default Python will compare memory addresses, since our class does not implement the `__eq__` method used for `==` comparisons.

In [ ]:
p1, p2

(Point(2,3), Point(2,3))

In [ ]:
p2 > p3

TypeError: '>' not supported between instances of 'Point' and 'Point'

So, that class does not support the comparison operators such as `<`, `<=`, etc.

Even `==` does not work as expected - it will use the memory address instead of using a comparison of the `x` and `y` coordinates as we might probably expect.

For the `<` operator, we need our class to implement the `__lt__` method, and for `==` we need the `__eq__` method.

Other comparison operators are supported by implementing a variety of functions such as `__le__` (`<=`), `__gt__` (`>`), `__ge__` (`>=`).

We are going to add the `__lt__` and `__eq__` methods to our Point class.

We will consider a Point object to be smaller than another one if it is closer to the origin (i.e. smaller magnitude).

In [ ]:
del Point

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented
        
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
        
    def __repr__(self):
        return '{0}({1},{2})'.format(self.__class__.__name__,self.x, self.y)

In [ ]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [ ]:
p1, p2, p1 == p2

(Point(2,3), Point(2,3), True)

In [ ]:
p2, p3, p2 == p3

(Point(2,3), Point(0,0), False)

As we can see, `==` now works as expected

In [ ]:
p4 = Point(1, 2)

In [ ]:
abs(p1), abs(p4), p1 < p4

(3.605551275463989, 2.23606797749979, False)

Great, so now we have `<` and `==`, does this mean Python magically implemented a `>` operator (i.e. not < and not ==)?

Not exactly! What happened is that since `p1` and `p4` are both points, running the comparison `p1 > p4` - and Python did do that automatically for us.

But it has not implemented any of the others, such as `>=` and `<=`:

In [ ]:
p1 <= p4

TypeError: '<=' not supported between instances of 'Point' and 'Point'

Now, although we could proceed in a similar way and define `>=`, `<=` and `>` using the same technique, observe that if `<` and `==` is defined then:

* `a <= b` if `a < b or a == b`
* `a > b` if `not(a < b)`
* `a >= b` if `not(a < b or a == b)`

So, to be quite generic we could create a decorator that will implement these last three operators as long as `==` and `<` are defined. We could then any class that implements just those two operators.


In [ ]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not(self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not(self < other)
    return cls

In reality, the code above is NOT a good implementation at all. We are not checking that the types are compatible and returning a `NotImplemented` result if appropriate. I am also using inline operators (`<` and `==`) instead of the dunder functions (`__lt__` and `__eq__`). I just kept it simple because we'll use a better alternative in a bit.

For example, a better way to implement `__ge__` would be as follows:

In [ ]:
def ge_from_lt(self, other):
    # self >= other if not (other < self)
    result = self.__lt__(other)
    if result is NotImplemented:
        return NotImplemented
    else:
        return not result

You may be wondering why I used `__lt__` instead of just using the `<` operator. This is because I want to actually look at the result of the operation without raising an exception if the operation is not implemented. The way I have the total ordering decorator implemented could c ause an infinite loop because when I evaluate `self < other`, if an exception is raised, Python will try to reflect that operation too, and we get into an infinite loop (which eventually terminates in a stack overflow). This was actually a bug in Python's standard library implementation of a `complete_ordering` decorator (called `total_ordering`) that was resolved in 3.4

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented
    
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
        
    def __repr__(self):
        return '{0}({1},{2}'.format(self.__class__, self.x, self.y)

In [ ]:
Point = complete_ordering(Point)

In [ ]:
p1, p2, p3 = Point(1, 1), Point(3, 4), Point(3, 4)

In [ ]:
abs(p1), abs(p2), abs(p3)

(1.4142135623730951, 5.0, 5.0)

In [ ]:
p1 < p2, p1 <= p2, p1 > p2, p1 >= p2, p2 > p2, p2 >= p3

(True, True, False, False, False, True)

Now the `complete_ordering` decorator can also be directly applied to any class that defines `__eq__` and `__lt__`

In [ ]:
@complete_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)
        
    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented
        
    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented

In [ ]:
g1 = Grade(10, 100)
g2 = Grade(20, 30)
g3 = Grade(5, 50)

In [ ]:
g1 <= g2, g1 == g3, g2 > g3

(True, True, True)

Often, given the `==` operator and just one of the other comparison operators (`<`, `<=`, `>`, `>=`), then all the rest can be derived.

Our decorator insisted on `==` and `<`. but we could make it better by insisting on `==` and any one of the other operators. This will of course make our decorator more complicated, and in fact, Python has this precise functionality built in to the, you guessed it, `functools` module!

It is a decorator called `total_ordering`

Let's see it in action:

In [ ]:
from functools import total_ordering

In [ ]:
@total_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)
     
    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented
    
    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented

In [ ]:
g1, g2 = Grade(80, 100), Grade(60, 100)

In [ ]:
g1 >= g2, g1 > g2

(True, True)

Or we could also do it this way:

In [ ]:
@total_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)
     
    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented
    
    def __gt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent > other.score_percent
        else:
            return NotImplemented

In [ ]:
g1, g2 = Grade(80, 100), Grade(60, 100)

In [ ]:
g1 >= g2, g1 > g2, g1 <= g2, g1 < g2

(True, True, False, False)

### 13 - Decorator Application - Single Dispatch

Consider an application where we want to provide similar functionality but that varies slightly depending on the argument types passed in.

In this set of examples we consider this problem where functionality differs based on a single argument's type (hence single disptch) instead of the type of multiple arguments (which would be multi dispatch)

If you have a background in some other OO languages such as Java or C#, you'll know that we can easily do something like this by basically **overloading** functions: using a different data type for the function parameter, hence changing the function signature. Then although the name of the function is the same, calling `do_something(100)` and `do_something('java')` would call a different function, the first one would call the `do_something(int)` function and the second would call the `do_something(String)` function.

Of course, Python is not statically typed, so even if Python had function overloading built-in, we would not be able to make such a distinction in our function signatures since there is nothing that says that a parameter must be of a specific type, so in best case scenario we would have to "distinguish" functions with the same name only by the number of parameters they take. And then we'd have to somehow deal with variable numbers of positional and keyword arguments too.. Uuugh! In any event, single dispatch could never work.

Instead we have to come up with a different solution.

Let's say we want to display various data types in html format, with different presentations for integers (we want both base 10 and hex values), floats (we always want it rounded to 2 decimal points), strings (we want the string html-escaped, and all newline characters replaced by ` `), lists and tuples should be implemented using bulleted lists, and the same with dictionaries except we want the name/value pair to be displayed in the bulleted list.

For starters, let's just implement individual functions to do each of those things.

I am going to keep the function very simple, but in practice you should handle situations like None objects, empty lists and dictionaries, possibly the wrong type being passed to the function, etc.

In [ ]:
from html import escape

def html_escape(arg):
    return escape(str(arg))
                      
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))
                                  
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')
                                  
def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item)) 
             for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'
                                  
def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k), html_escape(v)) 
             for k, v in d.items())    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
print(html_str("""this i
a multi line string
with special characters: 10 < 100"""))

this i<br/>
a multi line string<br/>
with special characters: 10 &lt; 100


In [ ]:
print(html_int(255))

255(<i>0xff</i)


In [ ]:
print(html_escape(3+10j))

(3+10j)


Ideally we would want to just have to cll a single function, maybe `htmlize` that would figure out which particular flavor of the `html_xxx` function to call dependig on the argument type.

We could try it as follows:

In [ ]:
from decimal import Decimal

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        # default behavior - just html escape string representation
        return html_escape(str(arg))

Now we can essentially use the same function call to handle different types - the `htmlize` function is a dispatcher - it dispatches the request to a different function based on the argument type. (There's a much better way to do some of this, but we'll have to wait until we cover abstract base classes to do so).

In [ ]:
print(htmlize([1, 2, 3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [ ]:
print(htmlize(dict(key1=1, key2=2)))

<ul>
<li>key1=1</li>
<li>key2=2</li>
</ul>


In [ ]:
print(htmlize(255))

255(<i>0xff</i)


But there are a number of shortcomings here:

In [ ]:
print(htmlize(["""first element is
a multi-line string""", (1, 2, 3)]))

<ul>
<li>first element is
a multi-line string</li>
<li>(1, 2, 3)</li>
</ul>


As you can see, the multi-line string did not get the newline characters replaced, the tuple was not rendered as an html list, and the integers do not have their hex representation.

So we just need to redefine the `html_list` and `html_dict` functions to use the `htmlize` function:

In [ ]:
def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(html_escape(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
print(htmlize(["""first element is 
a multi-line string""", (1, 2, 3)]))

<ul>
<li>first element is <br/>
a multi-line string</li>
<li><ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul></li>
</ul>


Much better, but hopefully you spotted something that might seem problematic!

Do we not have a circular reference?

In order to define `html_list` and `html_dict` we needed to call `htmlize`, but in order to define `htmlize` we needed to call `html_list` and `html_dict`.

Remember that in Python we can reference a function inside the body of another function before the function has been defined, as long as by the time we call the first function, the second one has been defined. So this is actually OK.

If you don't believe me and want to make sure of this yourself, go ahead and reset your Kernel (click on the Kernel | Restart menu option), and run the following code without running anything prior to this.

The `htmlize` function body makes calls to other functions such as `html_escape`, `html_int`, etc that have not actually been defined yet.

In [ ]:
from html import escape
from decimal import Decimal

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple) or isinstance(arg, set):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        # default behavior - just html escape string representation
        return html_escape(str(arg))

Now we define all the functions that `htmlize` uses before we actually call `htmlize` and all is good:

In [ ]:
def html_escape(arg):
    return escape(str(arg))
                      
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))
                                  
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')
                                  
def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'
                                  
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(html_escape(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
print(htmlize(["""first element is 
a multi-line string""", (1, 2, 3)]))

<ul>
<li>first element is <br/>
a multi-line string</li>
<li><ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul></li>
</ul>


As you can see this works just fine.

But we still have something undesirable. You'll notice that the dispatch function `htmlize` needs to have this big `if...elif...else` statement that will just keep growing as we need to handle more and more types (including potentially custom types).

This will just get unwieldy, and not very flexible (every time someone creates a new type that has to have a special html representation they will need to go into the `htmlize` function and modify it.

So instead, we are going to try a more flexible approach using decorators.

The way we are going to approach this is to create a dispatcher function, and then separately "register" each type-specific function with the dispatcher.

First, we are going to create a decorator that will do something that may seem kind of silly - it is going to take the decorated function and store it in a dictionary, using a key consisting of the **type** `object`.

Then when the returned closure is called, the closure will call the function stored in that dictionary.

In [ ]:
def singledispatch(fn):
    registry = dict()
    registry[object] = fn
    
    def inner(arg):
        return registry[object](arg)
    
    return inner

In [ ]:
@singledispatch
def htmlizer(arg):
    return escape(str(arg))

In [ ]:
htmlizer('a < 10')

'a &lt; 10'

Next, we are going to add some functions to that `registry` dictionary, and modify our inner function to choose the correct function from the registry, or pick a default based on the type of the argument:

In [ ]:
def singledispatch(fn):
    registry = dict()
    
    registry[object] = fn
    registry[int] = lambda arg: '{0}(<i>{1}</i)'.format(arg, str(hex(arg)))
    registry[float] = lambda arg: '{0:.2f}'.format(round(arg, 2))
    
    def inner(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    return inner

In [ ]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [ ]:
htmlize(10)

'10(<i>0xa</i)'

In [ ]:
htmlize(3.1415)

'3.14'

Now, we want a way to add the specialized functions to the `registry` dictionary from outside the `singledispatch` function - to do so we will create a parametrized decorator that will (1) take the type as a parameter, and (2) return a closure that will decorate the function associated with the type:

In [ ]:
def singledispatch(fn):
    registry = dict()
    
    registry[object] = fn
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
        return inner
        
    
    def decorator(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    
    return decorator

But of course this is not good enough - how do we get a hold of the `register` function from outside `singledispatch`? Remember, `singledispatch` is a decorator that returns the `decorated` closure, not the `register` closure.

We can do this by adding the `register` function as an **attribute** of the `decorated` function before we return it.

While we're at it we're also going to:

* add the `registry` dictionary as an attribute as so we can look into it to see what it contains.

* add another function that given a type will return the function associated with that type (or the default function if the type is not found in the dictionary)

In [ ]:
def singledispatch(fn):
    registry = dict()
    
    registry[object] = fn
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn  # we do this so we can stack register decorators!
        return inner
   
    def decorator(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    
    def dispatch(type_):
        return registry.get(type_, registry[object])

    decorator.register = register
    decorator.registry = registry.keys()
    decorator.dispatch = dispatch
    return decorator

In [ ]:
@singledispatch
def htmlize(arg):
    return escape(str(arg))

And we can see that `htmlize` (that returned `inner`) function has an attribute called `register`:

In [ ]:
htmlize.register

<function __main__.singledispatch.<locals>.register(type_)>

as well as that `registry` attribute that we put in just we could see what keys are in the `registry` dictionary:

In [ ]:
htmlize.registry

dict_keys([<class 'object'>])

We can also ask it what function it is going to use for any specific type (currently we only have one registered, the default, for the most general `object` type):

In [ ]:
htmlize.dispatch(str)

<function __main__.htmlize(arg)>

And you'll note that the extended scope of `register` and `dispatch` is the same as the extended scope of `htmlize`.

So now we can register some functions (it will store the function with associated data type in the `registry` dictionary):

In [ ]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

We can peek into the registered types:

In [ ]:
htmlize.registry

dict_keys([<class 'object'>, <class 'int'>])

and we can ask the decorated `htmlize` function what function it is going to use for the `int` type:

In [ ]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

and we can actually call it as well:

In [ ]:
htmlize(100)

'100(<i>0x64</i)'

The huge advantage now is that we can keep registering new handlers from anywhere in our module, or even from outside our module!

In [ ]:
@htmlize.register(float)
def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(htmlize(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
htmlize.registry

dict_keys([<class 'object'>, <class 'int'>, <class 'float'>, <class 'str'>, <class 'list'>, <class 'tuple'>, <class 'dict'>])

In [ ]:
print(htmlize([1, 2, 3]))

<ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul>


In [ ]:
print(htmlize((1, 2, 3)))

<ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul>


In [ ]:
print(htmlize("""this
is a multi line string with
a < 10"""))

this<br/>
is a multi line string with<br/>
a &lt; 10


Our single dispatch decorator works quite well - but it has some limitations. For example it cannot handle functions that take in more than one argument (in which case dispatching would be based on the type of the first argument), and we also are not allowing for types based on parent classes - for example, integers and booleans are both integral numbers - i.e. they both inherit from the Integral base class. Similarly lists and tuples are both more generic Sequence types. We'll see this in more detail when we get to the topic of abstract base classes (ABC's).

In [ ]:
from numbers import Integral

In [ ]:
isinstance(100, Integral)

True

In [ ]:
isinstance(True, Integral)

True

In [ ]:
isinstance(100.5, Integral)

False

In [ ]:
type(100) is Integral

False

In [ ]:
type(True) is Integral

False

In [ ]:
(100).__class__

int

In [ ]:
(True).__class__

bool

The way we have implement our decorator, if we register an Integral generic function, it won't pick up either integers or Booleans.

We can certainly fix this shortcoming ourselves, but of course...

We can can use Python's built-in single dispatch support, in ...

you guessed it!

the `functools` module.

In [ ]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence

In [ ]:
@singledispatch
def htmlize(a):
    return escape(str(a))

The `singledispatch` returned closure has a few attributes we can use:

1. A `register` decorator (just like ours did)
2. A `registry` property that is the registry dictionary
3. A `dispatch` function that can be used to determine which registry key (registered type) it will use for the specified type.

In [ ]:
@htmlize.register(Integral)
def htmlize_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a))) 

In [ ]:
htmlize.dispatch(int)

<function __main__.htmlize_int(a)>

In [ ]:
htmlize.dispatch(bool)

<function __main__.htmlize_int(a)>

In [ ]:
htmlize(100)

'100(<i>0x64</i)'

In [ ]:
htmlize(True)

'True(<i>0x1</i)'

In [ ]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
htmlize.dispatch(list)

<function __main__.html_sequence(l)>

In [ ]:
htmlize.dispatch(tuple)

<function __main__.html_sequence(l)>

In [ ]:
htmlize.dispatch(str)

<function __main__.html_sequence(l)>

You'll note that a string is also a sequence type, hence our dispatcher will call the `html_sequence` function on a string.

In fact, at this point things would not even run properly.

If we were to call

`htmlize('abc')`

we'd get an infinite recursion!

The call to `htmlize` the string abc would treat it as a sequence, which would call `htmlize` character by character. But each character is itself just a string of length 1, so it will `htmlize` for that single character, which would treat it as a sequence, which would call `htmlize` for that single character again, and so on, in an infinite loop.

In [ ]:
# htmlize('abc')

Instead, we are going to register a string handler specifically - that way we will avoid that problem entirely:

In [ ]:
@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

In [ ]:
htmlize.dispatch(str)

<function __main__.html_str(s)>

So, even though a string is both an `str` instance and in general a sequence type, the "closest" type will be picked by the dispatcher (again something our own implementation did not do).

This means, we have something for generic sequences, but something specific for more specialized strings.

In [ ]:
htmlize('abc')

'abc'

We can do the same thing with sequences - right now `html_sequence` will be used for both lists and tuples.

But suppose we want slightly different handling of tuples:

In [ ]:
@htmlize.register(tuple)
def html_tuple(t):
    items = [escape(str(item)) for item in t]
    return '({0})'.format(', '.join(items))

In [ ]:
htmlize.dispatch(list)

<function __main__.html_sequence(l)>

In [ ]:
htmlize.dispatch(tuple)

<function __main__.html_tuple(t)>

In [ ]:
print(htmlize(['a', 100, 3.14]))

<ul>
<li>a</li>
<li>100(<i>0x64</i)</li>
<li>3.14</li>
</ul>


In [ ]:
print(htmlize(('a', 100, 3.14)))

(a, 100, 3.14)


One thing of note is that we started our decoration with a `@singledispatch` decorator - you'll notice that no specific type was indicated here - and in fact this means the dispatcher will use the generic `object` type.

This means that any object type not specifically handled by our dispatcher will fall back on that `object` key - hence you can think of it as the default for the dispatcher.

In [ ]:
type(None)

NoneType

In [ ]:
htmlize.dispatch(type(None))

<function __main__.htmlize(a)>

In [ ]:
type(1+1j)

complex

In [ ]:
htmlize.dispatch(complex)

<function __main__.htmlize(a)>

In [ ]:
type(3)

int

In [ ]:
htmlize.dispatch(int)

<function __main__.htmlize_int(a)>

Lastly, because the name of the individual specialized functions does not really matter to us (the dispatcher will pick the appropriate function), it is quite common for an underscore character ( _ ) to be used for the function name - the memory address of each specialized function will be stored in the `registry` dictionary, and the function name does not matter - in fact we can even add lambdas to the registry.

In [ ]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [ ]:
@htmlize.register(int)
def _(a):
    return '{0}({1})'.format(a, str(hex(a)))

In [ ]:
@htmlize.register(str)
def _(s):
    return escape(s).replace('\n', '<br/>\n')

In [ ]:
htmlize.register(float)(lambda f: '{0:.2f}'.format(f))

<function __main__.<lambda>(f)>

In [ ]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              int: <function __main__._(a)>,
              str: <function __main__._(s)>,
              float: <function __main__.<lambda>(f)>})

But note that the `__main__` function for `int` and `str` are not the same functions (even tough they have the same name):

In [ ]:
id(htmlize.registry[str])

1954621336160

In [ ]:
id(htmlize.registry[int])

1954650658128

And everything works as expected:

In [ ]:
htmlize(100)

'100(0x64)'

In [ ]:
htmlize(3.1415)

'3.14'

In [ ]:
print(htmlize("""this
is a multi-line string
a < 10"""))

this<br/>
is a multi-line string<br/>
a &lt; 10


If this same name but different function thing has you confused, look at it this way:

In [ ]:
def my_func():
    print('my_func initial')

In [ ]:
id(my_func)

1954650516208

In [ ]:
f = my_func

In [ ]:
id(f)

1954650516208

So, `f` and `my_func` point to the same function in memory.

Let's go ahead and "redefine" the function `my_func`:

In [ ]:
def my_func():
    print('second my_func')

In fact, we did not "redefine" the previous `my_func`, it still exists in memory (and `f` still points to it). Instead we have re-assigned the function that `my_func` points to:

In [ ]:
id(my_func)

1954650656976

But the original `my_func` is still around, and 'f' still has a reference to it:

In [ ]:
id(f)

1954650516208

So, we can call each one:

In [ ]:
f()

my_func initial


In [ ]:
my_func()

second my_func


But the function `__name__` have the same value:

In [ ]:
f.__name__

'my_func'

In [ ]:
my_func.__name__

'my_func'

Just always keep in mind that labels point to something in memory, it is not the object itself. So in this case we have two distinct objects (functions) which happen to have the same name, but are two very different objects - `f` points to the first one we created, and `my_func` points to the second.